<a href="https://colab.research.google.com/github/wtburdon/AI-Pokemon-Identifier/blob/main/PokemonIdentifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Learning Pytorch while making a Pokemon identifier AI!**

## **Setting Up**

### **Imports**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms # For image recognition
from torchvision.datasets import ImageFolder
import timm

# This is all data visualization stuff
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Accesses dataset (Dataset used : https://www.kaggle.com/datasets/hlrhegemony/pokemon-image-dataset)
from google.colab import drive
drive.mount('/content/drive')

### **Creating Dataset class**

In [ ]:
class PokemonDataset(Dataset):
  def __init__(self, data_dir, transform = None):
    self.data = ImageFolder(data_dir, transform = transform)


  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

  @property
  def classes(self):
    return self.data.classes

### **Initialize Dataset**

In [45]:
#Resize images and then turn them into tensors
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

data_dir = '/content/drive/MyDrive/PokemonData/images'

dataset = PokemonDataset(data_dir, transform)

In [40]:
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class) # Checking to make sure all of the pokemon are loaded correctly to the dictionary

{0: 'Abomasnow', 1: 'Abra', 2: 'Absol', 3: 'Accelgor', 4: 'Aegislash', 5: 'Aerodactyl', 6: 'Aggron', 7: 'Aipom', 8: 'Alakazam', 9: 'Alcremie', 10: 'Alomomola', 11: 'Altaria', 12: 'Amaura', 13: 'Ambipom', 14: 'Amoonguss', 15: 'Ampharos', 16: 'Anorith', 17: 'Appletun', 18: 'Applin', 19: 'Araquanid', 20: 'Arbok', 21: 'Arcanine', 22: 'Arceus', 23: 'Archen', 24: 'Archeops', 25: 'Arctovish', 26: 'Arctozolt', 27: 'Ariados', 28: 'Armaldo', 29: 'Aromatisse', 30: 'Aron', 31: 'Arrokuda', 32: 'Articuno', 33: 'Audino', 34: 'Aurorus', 35: 'Avalugg', 36: 'Axew', 37: 'Azelf', 38: 'Azumarill', 39: 'Azurill', 40: 'Bagon', 41: 'Baltoy', 42: 'Banette', 43: 'Barbaracle', 44: 'Barboach', 45: 'Barraskewda', 46: 'Basculin', 47: 'Bastiodon', 48: 'Bayleef', 49: 'Beartic', 50: 'Beautifly', 51: 'Beedrill', 52: 'Beheeyem', 53: 'Beldum', 54: 'Bellossom', 55: 'Bellsprout', 56: 'Bergmite', 57: 'Bewear', 58: 'Bibarel', 59: 'Bidoof', 60: 'Binacle', 61: 'Bisharp', 62: 'Blacephalon', 63: 'Blastoise', 64: 'Blaziken', 65: 

In [47]:
img, label = dataset[500]
img.shape # converts tensor to show the size

Image: tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), Label: 186


torch.Size([3, 128, 128])

### **DataLoader Initialization**

In [54]:
dataloader = DataLoader(dataset, batch_size = 32, shuffle = True)

In [55]:
for imgs, labels in dataloader:
  break

Image: tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), Label: 343
Image: tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [

In [58]:
imgs.shape
labels

tensor([343, 459, 765, 126, 311, 225, 452, 861, 658, 724, 424, 523, 818, 878,
        800,  11, 402, 734, 379, 771, 479, 787,  21, 756, 414, 837, 107, 896,
        617, 447, 421, 799])

## **Pytorch Model**